In [1]:
#Primero importamos las librerías que vamos a usar
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OneHotEncoder,OrdinalEncoder

import folium
import plotly.express as px

In [2]:
#Después importamos los datos del archivo CSV que tenemos
df_bmw = pd.read_csv("/Users/fernandopradagorge/Documents/Universidad/Master Data Science and AI/Nuclio Projects/Entregables/Entregable_1_15ENE24/bmw_pricing_v3.csv")

In [46]:
df_bmw.head()

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,NaN,True,True,False,NaN,True,NaN,True,NaN,11300.0,2018-01-01
1,BMW,M4,13929.0,317.0,NaN,petrol,grey,convertible,True,True,False,NaN,False,True,True,True,69700.0,2018-02-01
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,NaN,False,False,False,NaN,True,False,True,False,10200.0,2018-02-01
3,BMW,420,128035.0,135.0,NaN,diesel,red,convertible,True,True,False,NaN,True,True,True,NaN,25100.0,2018-02-01
4,BMW,425,97097.0,160.0,NaN,diesel,silver,NaN,True,True,False,False,False,True,True,True,33400.0,2018-04-01


In [3]:
#Una vez comprobado que el DataFrame carga bien, vamos a ver la estructura que tienen los datos.
df_bmw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   marca                        3873 non-null   object 
 1   modelo                       4840 non-null   object 
 2   km                           4841 non-null   float64
 3   potencia                     4842 non-null   float64
 4   fecha_registro               2420 non-null   object 
 5   tipo_gasolina                4838 non-null   object 
 6   color                        4398 non-null   object 
 7   tipo_coche                   3383 non-null   object 
 8   volante_regulable            4839 non-null   object 
 9   aire_acondicionado           4357 non-null   object 
 10  camara_trasera               4841 non-null   object 
 11  asientos_traseros_plegables  1452 non-null   object 
 12  elevalunas_electrico         4841 non-null   object 
 13  bluetooth         

In [4]:
#Cambiamos las columnas fecha_registro y fecha_venta a formato datetime
for i in ["fecha_registro","fecha_venta"]:
    df_bmw[i]=pd.to_datetime(df_bmw[i])

df_bmw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   marca                        3873 non-null   object        
 1   modelo                       4840 non-null   object        
 2   km                           4841 non-null   float64       
 3   potencia                     4842 non-null   float64       
 4   fecha_registro               2420 non-null   datetime64[ns]
 5   tipo_gasolina                4838 non-null   object        
 6   color                        4398 non-null   object        
 7   tipo_coche                   3383 non-null   object        
 8   volante_regulable            4839 non-null   object        
 9   aire_acondicionado           4357 non-null   object        
 10  camara_trasera               4841 non-null   object        
 11  asientos_traseros_plegables  1452 non-null 

In [5]:
#Para una mejor lectura y comprensión de los datos, dividiremos la columna fecha_venta en 3 nuevas según el día, el mes y  el año de venta.
for col in ["fecha_venta"]:
    df_bmw[col + "_MES"] = df_bmw[col].dt.month
    df_bmw[col + "_AÑO"] = df_bmw[col].dt.year

df_bmw.head()

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta,fecha_venta_MES,fecha_venta_AÑO
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,NaN,True,True,False,NaN,True,NaN,True,NaN,11300.0,2018-01-01,1.0,2018.0
1,BMW,M4,13929.0,317.0,NaT,petrol,grey,convertible,True,True,False,NaN,False,True,True,True,69700.0,2018-02-01,2.0,2018.0
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,NaN,False,False,False,NaN,True,False,True,False,10200.0,2018-02-01,2.0,2018.0
3,BMW,420,128035.0,135.0,NaT,diesel,red,convertible,True,True,False,NaN,True,True,True,NaN,25100.0,2018-02-01,2.0,2018.0
4,BMW,425,97097.0,160.0,NaT,diesel,silver,NaN,True,True,False,False,False,True,True,True,33400.0,2018-04-01,4.0,2018.0


In [6]:
#Ahora procederemos a quitar las columnas de marca, modelo (por ser irrelevantes para el estudio) y fecha venta por ser repetida:
df_bmw.drop(["modelo", "marca", "fecha_venta"], axis=1, inplace=True)

In [9]:
#Vamos a buscar los nulos y a quitarlos de enmedio
sin_nulos = ["km", "potencia", "tipo_gasolina", "volante_regulable", "camara_trasera", "elevalunas_electrico", "precio", "fecha_venta_MES", "fecha_venta_AÑO"]
df_bmw2 = df_bmw.dropna(subset=sin_nulos)

df_bmw2.isnull().sum()

km                                0
potencia                          0
fecha_registro                 2411
tipo_gasolina                     0
color                           442
tipo_coche                     1451
volante_regulable                 0
aire_acondicionado              482
camara_trasera                    0
asientos_traseros_plegables    3376
elevalunas_electrico              0
bluetooth                       726
gps                               0
alerta_lim_velocidad            726
precio                            0
fecha_venta_MES                   0
fecha_venta_AÑO                   0
dtype: int64

In [10]:
#Y ahora guardamos una copia en df_bmw3
df_bmw3= df_bmw2.copy()

In [29]:
#Guardamos el DataFrame en  un pickle para seguir trabajando desde aquí mas adelante y no tener que cargar todo de nuevo
pd.to_pickle(df_bmw3, "/Users/fernandopradagorge/Documents/Universidad/Master Data Science and AI/Nuclio Projects/Entregables/Entregable_1_15ENE24/primera_entrega_15ENE24_PK")

In [28]:
#Me salen algunos outliers en la columna "km" en un primer vistazo por lo que procedo a su eliminación para que los datos estén mas normalizados

df_bmw3.drop(df_bmw3[((df_bmw3["km"])<0 | (df_bmw3["km"]>400000))].index, inplace=True)